![Codenation](https://forum.codenation.com.br/uploads/default/original/2X/2/2d2d2a9469f0171e7df2c4ee97f70c555e431e76.png)

__Autor__: Kazuki Yokoyama (kazuki.yokoyama@ufrgs.br)

# _Feature engineering_

![cover](https://venturebeat.com/wp-content/uploads/2018/07/feature_engineering.jpg?resize=680%2C198&strip=all)

Neste módulo, trabalharemos a engenharia de _features_, que consiste em preparar os nossos dados para alimentar os algoritmos de ML adequadamente. Ao contrário do mundo dos tutoriais, na vida real os dados dificilmente estarão prontos para serem consumidos. Grande parte do tempo de um projeto de ML é gasto com a engenharia de _features_, e quanto melhor a qualidade desta etapa, maiores são as chances de melhores resultados nas etapas seguintes.

## Importação das bibliotecas

In [ ]:
import functools
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as sct
import seaborn as sns
from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import (
    CountVectorizer, TfidfTransformer, TfidfVectorizer
)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, Binarizer, KBinsDiscretizer,
    MinMaxScaler, StandardScaler, PolynomialFeatures
)

In [ ]:
# Algumas configurações para o matplotlib.
%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 12)

sns.set()

In [ ]:
np.random.seed(1000)

## _One-hot encoding_

Até aqui, nós praticamente ignoramos a existência de variáveis categóricas. Focamos nas variáveis numéricas porque elas são simples de lidar e bastante comuns. Ainda assim, variáveis categóricas são encontradas facilmente e precisamos de uma forma de trabalhar com elas.

Uma das formas mais simples de representação de variáveis categóricas é através do método chamado _one-hot enconding_. Com ele, uma variável categórica com $h$ categorias é transformada em $h$ novas variáveis binárias (0 ou 1), onde a presença do 1 (_hot_) significa que aquela observação pertence àquela categoria, e 0 (_cold_) que não pertence. Veja um exemplo abaixo:

In [ ]:
rows = 100

height = np.random.normal(loc=1.70, scale=0.2, size=rows).round(3)
score = np.random.normal(loc=7, scale=1, size=rows).round(2)
courses = ["Math", "Physics", "Biology"]
course = np.random.choice(courses, size=rows)

data = pd.DataFrame({"Height": height, "Score": score, "Course": course})

data.head(10)

Criamos um _data set_ que contém duas variáveis numéricas (`Height` e `Score`) e uma variável categórica (`Course`). Nosso objetivo com o _one-hot encoding_ é transformar a variável `Course` em uma sequência de variáveis numéricas binárias, cada uma descrevendo uma classe da variável. Neste caso, como temos três categorias para `Course` (Biology, Physics e Math), teremos três novas variáveis binárias.

Vamos treinar esse _encoder_:

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False, dtype=np.int)

#one_hot_encoder.fit(data[["Course"]])

#course_encoded = one_hot_encoder.transform(...)

course_encoded = one_hot_encoder.fit_transform(data[["Course"]])

course_encoded[:10]

A saída é um `np.ndarray` com formato `(n, h)`, onde `n` é o número de observações no _data set_ e `h` é o número de categorias da variável codificada.

In [ ]:
course_encoded.shape

No atributo `categories_` do _encoder_, temos as categorias da variável:

In [ ]:
one_hot_encoder.categories_

Podemos criar as novas colunas que descrevem cada categoria. Repare que, para qualquer linha, apenas uma das colunas contém um 1, indicando a qual categoria aquela observação pertence. Isso acontece, obviamente, se as categorias forem mutuamente exclusivas (uma observação não pode pertencer a mais de uma categoria simultaneamente).

In [ ]:
columns_encoded = one_hot_encoder.categories_[0]

data_encoded = pd.concat([data, pd.DataFrame(course_encoded, columns=columns_encoded)], axis=1)

data_encoded.head(10)

Como você deve imaginar, a maior parte da matriz retornada é composta por zeros, sendo apenas alguns elementos compostos de um. Dizemos que essa matriz é __esparsa__. É um grande desperdício de memória trabalhar diretamente como uma matriz esparsa assim. Por isso, o _default_ do `OneHotEncoder` é retornar uma `sparse matrix` do NumPy, economizando espaço em memória:

In [ ]:
one_hot_encoder_sparse = OneHotEncoder(sparse=True) # sparse=True é o default.

course_encoded_sparse = one_hot_encoder_sparse.fit_transform(data[["Course"]])

course_encoded_sparse

Para acessar os dados dessa matriz, podemos convertê-la para um _array_ não esparso:

In [ ]:
course_encoded_sparse.toarray()[:10]

## Binarização (_Binarization_)

Binarização é o processo de discretizar uma variável numérica em dois níveis com base em um _threshold_. Isso pode ser útil, por exemplo, para tornar uma variável numérica contínua em uma variável binária alvo de duas classes (positiva ou negativa).

No exemplo abaixo, vamos separar a variável `Height` em dois grupos, utilizando 1.80 m como _threshold_ de separação. Observações que possuam menos de 1.80 m terão valor 0, enquanto aquelas com mais de 1.80 m terão valor 1:

In [ ]:
tall = (data_encoded.Height > 1.80)

tall[:10]

In [ ]:
binarizer = Binarizer(threshold=1.80).fit(data_encoded[["Height"]])

height_binary = binarizer.transform(data_encoded[["Height"]])

height_binary[:10]

O `Binarizer` tem como saída uma matriz binária numérica. Podemos transformá-la em um vetor de _bool_:

In [ ]:
height_bool = pd.DataFrame(height_binary.flatten().astype(bool), columns=["Tall"])

height_bool.head(10)

Vamos adicionar a nova variável `Tall`, que indica se a pessoa é alta (> 1.80 m), ao nosso _data set_:

In [ ]:
data_encoded = pd.concat([data_encoded, height_bool], axis=1)

data_encoded.head(10)

## Discretização (_Binning_)

Discretização, como o nome diz, é o processo de discretizar ou separar em intervalos contínuos uma variável numérica. Isso pode ser útil para converter uma variável numérica em categórica, quando o valor exato numérico não for tão importante quanto o intervalo onde ele se encontra.

Podemos criar _bins_ (_buckets_ ou intervalos) que contenham aproximadamente a mesma quantidade de observações, utilizando a estratégia `quantile` ou que sejam igualmente espaçados com a estratégia `uniform`.

No exemplo a seguir, criamos quatro intervalos da variável `Score` com a estratégia `quantile`:

In [ ]:
discretizer = KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="quantile")

discretizer.fit(data_encoded[["Score"]])

score_bins = discretizer.transform(data_encoded[["Score"]])

score_bins[:10]

Os limites dos intervalos estão disponíveis no atributo `bin_edges_`. Isso pode ser útil para criarmos _labels_ para colunas do _data set_ por exemplo:

In [ ]:
discretizer.bin_edges_

A função `get_interval()` abaixo facilita a criação de _labels_ indicativas dos intervalos:

In [ ]:
def get_interval(bin_idx, bin_edges):
  return f"{np.round(bin_edges[bin_idx], 2):.2f} ⊢ {np.round(bin_edges[bin_idx+1], 2):.2f}"

Cada um dos intervalos mostrados abaixo deve possuir aproximadamente a mesma quantidade de observações:

In [ ]:
bin_edges_quantile = discretizer.bin_edges_[0]

print(f"Bins quantile")
print(f"interval: #elements\n")
for i in range(len(discretizer.bin_edges_[0])-1):
    print(f"{get_interval(i, bin_edges_quantile)}: {sum(score_bins[:, 0] == i)}")

A _Series_ abaixo mostra alguns dos intervalos para os quais as observações foram encaixadas:

In [ ]:
score_intervals = pd.Series(score_bins.flatten().astype(np.int)).apply(get_interval, args=(bin_edges_quantile,))

score_intervals.head(10)

Também podemos criar uma nova variável, `Score_interval`, no nosso _data set_ com os intervalos (que agora são categorias):

In [ ]:
data_encoded = pd.concat([data_encoded, pd.DataFrame(score_intervals, columns=["Score_interval"])], axis=1)

data_encoded.head(10)

Como dito, podemos utilizar a estratégia `uniform` para criar _bins_ igualmente espaçados, independente do número de observações que cada um possui. Também podemos especificar o tipo de codificação utilizada. No caso a seguir, utilizamos `encode=onehot-dense` para informar que queremos que a saída seja codificada como o _one-hot encode_ visto anteriormente:

In [ ]:
discretizer_uniform = KBinsDiscretizer(n_bins=4, encode="onehot-dense", strategy="uniform")

discretizer_uniform.fit(data_encoded[["Score"]])

score_bins_uniform = discretizer_uniform.transform(data_encoded[["Score"]]).astype(np.int)

score_bins_uniform[:10]

Note como agora os intervalos são ligeiramente diferentes:

In [ ]:
bin_edges_uniform = discretizer_uniform.bin_edges_[0]

bin_edges_uniform

In [ ]:
score_intervals_columns = [get_interval(i, bin_edges_uniform) for i in range(4)]

print(f"Bins uniform")
print(f"interval: #elements\n")
for i in range(len(discretizer_uniform.bin_edges_[0])-1):
    print(f"{get_interval(i, bin_edges_uniform)}: {sum(score_bins_uniform[:, i])}")

Podemos adicionar as novas variáveis binárias no _data set_:

In [ ]:
data_encoded = pd.concat([data_encoded, pd.DataFrame(score_bins_uniform, columns=score_intervals_columns)], axis=1)

data_encoded.head(10)

## Normalização (_Scaling_)

Normalização é o processo de colocar uma variável numérica em uma escala pré-determinada, geralmente $[0, 1]$, mas também é comum ser $[-1, 1]$.

Para colocar no intervalo $[0, 1]$, basta subtrair cada valor da valor mínimo e dividir pela diferença do valor máximo e mínimo:

$$x_{\text{scaled}} = \frac{x - x_{\text{min}}}{x_{\text{max}} - x_{\text{min}}}$$

Abaixo, escalamos a variável `Score` no intervalo $[0, 1]$:

In [ ]:
minmax_scaler = MinMaxScaler(feature_range=(0, 1)) # Default feature_scale é (0, 1).

minmax_scaler.fit(data_encoded[["Score"]])

score_normalized = minmax_scaler.transform(data_encoded[["Score"]])

score_normalized[:10]

In [ ]:
score_normalized.min(), score_normalized.max()

Adicionamos a variável `Score` normalizada ao nosso _data set_:

In [ ]:
data_encoded = pd.concat([data_encoded, pd.DataFrame(score_normalized.flatten(), columns=["Score_normalized"])], axis=1)

data_encoded.head(10)

Para avaliar se os valores encontrados conferem, podemos utilizar a função `normalize` abaixo:

In [ ]:
def normalize(x, xmin, xmax):
  return (x - xmin)/(xmax - xmin)

A função `partial()` do módulo `functools` (_builtin_ do Python) permite "congelar" alguns parâmetros da função passaga como argumento, facilitando a invocação desta função quando tais parâmetros são constantes. No caso abaixo, "congelamos" os argumentos `xmin` e `xmax` da função `normalize()` com os valores mínimo e máximo da variável `Score`, respectivamente. Nas invocações subsequentes de `normalize` não precisaremos passar esses argumentos, somente o argumento "não congelado" `x`:

In [ ]:
normalize_score = functools.partial(normalize,
                                    xmin=data_encoded.Score.min(),
                                    xmax=data_encoded.Score.max())

O valor abaixo realmente confere com aquele encontrado pelo `MinMaxScaler`:

In [ ]:
normalize_score(data_encoded.Score[0]).round(6)

## Padronização (_Standardization_)

Padronização é o processo de tornar a variável com média zero e variância um. Esse processo não deve ser confundido com a normalização descrita acima.

O processo é simples, basta subtrair a média dos dados de cada observação e dividi-los pelo desvio-padrão:

$$x_{\text{standardized}} = \frac{x - \bar{x}}{s}$$

onde $\bar{x}$ indica a média amostral e $s$ o desvio-padrão amostral.

No exemplo abaixo, padronizamos a variável `Score`:

In [ ]:
standard_scaler = StandardScaler()

standard_scaler.fit(data_encoded[["Score"]])

score_standardized = standard_scaler.transform(data_encoded[["Score"]])

score_standardized[:10]

E adicionamos a variável padronizada ao nosso _data set_:

In [ ]:
data_encoded = pd.concat([data_encoded, pd.DataFrame(score_standardized.flatten(), columns=["Score_standardized"])], axis=1)

data_encoded.head(10)

Note que, ao contrário da variável normalizada, é possível ter valores negativos e positivos, menores e maiores que um. Isso é bem óbvio, pois os dados agora têm média 0 e variância 1:

In [ ]:
data_encoded.Score_standardized.mean(), data_encoded.Score_standardized.var()

Novamente, para avaliar os resultados obtidos, podemos escrever nossa própria função de padronização:

In [ ]:
def standardize(x, xmean, xstd):
  return (x - xmean)/xstd

In [ ]:
standardize_score = functools.partial(standardize,
                                      xmean=data_encoded.Score.mean(),
                                      xstd=data_encoded.Score.std())

Como esperado, o valor confere com o encontrado:

In [ ]:
standardize_score(data_encoded.Score[0])

## Criando um _Pipeline_

Todo esse processo de transformar os dados pode ser bastante trabalhoso e entendiante. Para facilitar as coisas, o sklearn dispõe de um mecanismo de _pipeline_ que funciona como ao esteira de uma linha de montagem. Cada etapa desse _pipeline_ é uma transformação nos dados, de forma que, ao final do _pipeline_, temos os dados totalmente transformados. A vantagem é que agora especificamos todas as etapas, ou transformações, de uma só vez, e podemos reaproveitar esse _pipeline_ no futuro.

In [ ]:
data.head(10)

Para evitar bagunçar com nosso _data set_ original, criamos uma cópia (rasa) dele:

In [ ]:
data_missing = data.copy()

E para tornar o exemplo mais interessante, adicionamos (ou removemos?) dados faltantes ao _data set_. Isso porque uma das transformações úteis que podemos aplicar no _pipeline_ é justamente a imputação de dados, ou seja, preencher dados faltantes.

As variáveis numéricas faltantes são representadas por `np.nan`, enquanto a variável categórica é representada pela classe `Unknown`:

In [ ]:
unknown_height_idx = pd.Index(np.random.choice(data_missing.index, 10, replace=False))
unknown_score_idx = pd.Index(np.random.choice(data_missing.index, 10, replace=False))
unknown_course_idx = pd.Index(np.random.choice(data_missing.index, 10, replace=False))

data_missing.loc[unknown_height_idx, "Height"] = np.nan
data_missing.loc[unknown_score_idx, "Score"] = np.nan
data_missing.loc[unknown_course_idx, "Course"] = "Unknown"

data_missing_idx = unknown_height_idx | unknown_score_idx | unknown_course_idx

data_missing.loc[data_missing_idx].head(10)

Criamos o _pipeline_ com as seguintes etapas:

1. Faça imputação dos dados, preenchendo os dados faltantes com a mediana dos dados presentes.
2. Faça a normalização dos dados no intervalo _default_ $[0, 1]$.
3. Crie novas variáveis através da expansão polinomial da variável original.

O `Pipeline` recebe uma lista de transformações representadas por tuplas de dois elementos. Cada tupla contém:

* O nome para a etapa (ou transformação ou estimador). Isso vai ser útil para recuperar algumas informações do _pipeline_ mais a frente.
* Um objeto da classe do transformador ou estimador, já com seus parâmetros configurados.

In [ ]:
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("minmax_scaler", MinMaxScaler()),
    ("poly_features", PolynomialFeatures(degree=2, include_bias=False))
])

Depois da especificação do nosso _pipeline_, podemos aplicá-lo simultaneamente a diversas variáveis (desde que as transformações especificadas façam sentido).

No exemplo abaixo, aplicamos esse _pipeline_ às variáveis `Height` e `Score` ao mesmo tempo:

In [ ]:
pipeline_transformation = num_pipeline.fit_transform(data_missing[["Height", "Score"]])

pipeline_transformation[:10]

Para ficar mais claro a saída do _pipeline_, podemos utilizar os nomes das _features_ geradas através do método `get_feature_names()`. Para tornar ainda mais claro, substituímos o que é chamado `x0` por `Height` e `x1` por `Score`, que é inferido pela ordem das variáveis no _pipeline_.

In [ ]:
poly_features = num_pipeline.get_params()["poly_features"].get_feature_names()
  
pipeline_columns = [old_name.replace("x0", "Height_n").replace("x1", "Score_n") for old_name in poly_features]

pipeline_columns

Criamos um novo _data set_ com essas variáveis resultantes do _pipeline_:

In [ ]:
height_score_normalized_poly = pd.DataFrame(pipeline_transformation, columns=pipeline_columns)

height_score_normalized_poly.head(6)

Podemos também criar outro _pipeline_ para a variável categórica `Course`. Como se trata de uma variável de natureza completamente diferente, precisamos especificar um _pipeline_ diferente com as seguintes transformações:

1. Preencha os dados faltantes (`None`) com a classe `Unknown`.
2. Crie novas variáveis binárias com o `OneHotEncoder`.

Assim como no _pipeline_ anterior, especificamos cada etapa como uma tupla com um nome e um objeto de um transformador ou estimador:

In [ ]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("one_hot_encoder", OneHotEncoder(sparse=False, dtype=np.int))
])

Após a especificação do _pipeline_, podemos aplicá-lo à nossa variável `Course`:

In [ ]:
course_pipeline_transformation = cat_pipeline.fit_transform(data_missing[["Course"]])

Agora, utilizaremos o nome que demos à etapa do `OneHotEncoder` para recuperar esse transformador através do método `get_params()`. Depois de recuperado o `OneHotEncoder`, acessamos seu atributo `categories_` (primeiro índice `[0]`, pois poderíamos ter aplicado o _pipeline_ a mais de uma variável categórica):

In [ ]:
course_columns = cat_pipeline.get_params()["one_hot_encoder"].categories_[0]

course_columns

Utilizamos a saída do _pipeline_ e os nomes das categorias recuperados do transformador para criar um novo `DataFrame`:

In [ ]:
course_discretized = pd.DataFrame(course_pipeline_transformation, columns=course_columns)

course_discretized.head(5)

Por fim, combinamos as saídas dos dois _pipelines_ para criar um único `DataFrame`:

In [ ]:
data_transformed = pd.concat([height_score_normalized_poly, course_discretized], axis=1)

data_transformed.head(10)

Vale ressaltar que:

* Poderíamos utilizar também o `ColumnTransformer` para compor (por isso, ele se encontra no módulo `sklearn.compose`) múltiplos `Pipeline` em diferentes variáveis.
* Os `Pipeline` não servem apenas para a transformação dos dados de treinamento. Eles também podem (e devem) ser usados para submeter os dados de teste e até de produção aos mesmos procedimentos dos dados de treinamento.

## _Outliers_

_Outliers_, os famosos "pontos fora da curva", são observações que não parecem seguir o mesmo padrão dos demais dados. Eles podem vir de distribuições diferentes, serem erros na coleta de dados, erros de medição etc.

Eles influenciam nossas análises e os nossos algoritmos ao apresentar comportamento distoante do resto do _data set_, impactando na média, variância, funções de perda e custo etc. Se fizer sentido, eles devem ser removidos ou transformados antes de prosseguirmos com a análise.

No entanto, devemos julgar com cautela sua remoção: __alguns _outliers_ são dados autênticos e devem ser estudados com atenção__. Por exemplo, a remoção de uma medição muito alta na temperatura de um reator seria um erro, pois essa medição pode estar nos indicando um potencial problema com o dispositivo.

Abaixo estudamos algumas técnicas simples para encontrar _outliers_.

![outlier](https://www.stats4stem.org/common/web/plugins/ckeditor/plugins/doksoft_uploader/userfiles/WithInfOutlier.png)

Começamos criando uma cópia da variável `Height` do nosso _data set_ para não impactar o original:

In [ ]:
height_outlier = data.Height.copy()

Adicionamos dez  _outliers_ que representam pessoas estranhamente baixas ou estranhamente altas para o padrão que estamos observando:

In [ ]:
height_outlier_idx = pd.Index(np.random.choice(height_outlier.index, 10, replace=False))

too_short_idx = pd.Index(height_outlier_idx[:5])
too_tall_idx = pd.Index(height_outlier_idx[5:])

height_outlier[too_short_idx] = np.random.normal(loc=1.30, scale=0.5, size=5)
height_outlier[too_tall_idx] = np.random.normal(loc=2.20, scale=0.5, size=5)

outlier_idx = too_short_idx | too_tall_idx

height_outlier[outlier_idx]


Note que nem todos dados gerados se tornaram realmente _outliers_. Como geramos de uma distribuição aleatória, corremos esse risco.

No entanto, temos alguns dados estranhos como 0.51 m e 2.73 m.

No _boxplot_ padrão, os dados mais extremos são mostrados como pontos fora do alcance dos _whiskers_ (as barrinhas do _box plot_).

No caso abaixo, notamos três pontos acima e três pontos abaixo do considerado "dentro da faixa normal".

In [ ]:
sns.boxplot(height_outlier, orient="vertical");

Uma primeira abordagem bem simples é encontrar os pontos do _box plot_ acima.

Tudo que estiver fora da faixa $[Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}]$ é considerado um ponto anômalo para aquele padrão:

In [ ]:
q1 = height_outlier.quantile(0.25)
q3 = height_outlier.quantile(0.75)
iqr = q3 - q1

non_outlier_interval_iqr = [q1 - 1.5 * iqr, q3 + 1.5 * iqr]

print(f"Faixa considerada \"normal\": {non_outlier_interval_iqr}")

Agora podemos identificar quais pontos encontram-se fora desse intervalo, ou seja, podem ser considerados _outliers_:

In [ ]:
outliers_iqr = height_outlier[(height_outlier < non_outlier_interval_iqr[0]) | (height_outlier > non_outlier_interval_iqr[1])]

outliers_iqr

Se estivermos seguos de que esses pontos representam de fato _outliers_ e que sua remoção não traz prejuízo à nossa análise, então podemos removê-los:

In [ ]:
height_no_outlier_iqr = height_outlier.drop(index=outliers_iqr.index)

Uma segunda abordagem é observar as estatísticas descritivas dos dados.

Repare no histograma abaixo:

In [ ]:
sns.distplot(height_outlier);

Dá para perceber que a maior parte dos dados concentra-se em torno da média (~ 1.7 m) e que apenas algumas observações encontram-se bastante distantes dela.

In [ ]:
height_outlier_mean = height_outlier.mean()
height_outlier_std = height_outlier.std()

height_outlier_mean, height_outlier_std

Um jeito de procurar por _outliers_ é ver quem se encontra fora do intervalo $[\bar{x} - k * \sigma, \bar{x} + k * \sigma]$, onde $k$ geralmente é 1.5, 2.0, 2.5 ou até 3.0.

Abaixo utilizamos o $k = 2$, pois esse valor faz sentido (alturas menores que 1.12 m ou maiores que 2.30 m fogem do nosso padrão):

In [ ]:
non_outlier_interval_dist = [height_outlier_mean - 2 * height_outlier_std, height_outlier_mean + 2 * height_outlier_std]

non_outlier_interval_dist

Novamente, conhecendo o intervalo, podemos identificar as observações que caem foram dele e removê-las:

In [ ]:
outliers_dist = height_outlier[(height_outlier < non_outlier_interval_dist[0]) | (height_outlier > non_outlier_interval_dist[1])]

outliers_dist

In [ ]:
height_no_outlier_dist = height_outlier.drop(index=outliers_dist.index)

Até agora, nossas métodos de identificação de _outlier_ foram baseadas em estatísticas descritivas do nosso _data set_ (quantis, média e variância). Porém, alguns testes de hipóteses também existem.

Um deles é o teste de Grubb. Esse é um teste bastante simples, cuja estatística de teste $G$ depende dos valores extremos do conjunto e da média amostral:

$$G = \frac{\vert x_{\text{\{min ou max\}}} - \bar{x}\vert}{s}$$

onde $\bar{x}$ é a média amostral e $s$ é o desvio-padrão da amostra.

A hipótese nula, $H_{0}$, é de que não existem _outliers_ no _data set_. O teste de Grubb assume que os dados originam-se de uma distribuição normal, então pode ser válido testar essa hipótese antes.

Rejeitamos a hipótese nula se o valor de $G$ encontrado for superior ao valor crítico do teste, que é dado por

$$G_{\text{crítico}} = \frac{n - 1}{\sqrt{n}} \sqrt{\frac{t_{\alpha',n-2}^{2}}{n - 2 + t_{\alpha',n-2}^{2}}}$$

onde $n$ é o tamanho da amostra, $t$ é um valor com distribuição t-Student e $\alpha'$ é $\alpha/2n$ se o teste for bilateral (procuramos _outliers_ muito acima ou muito abaixo) ou $\alpha/n$ se o teste for unilateral (acreditamos que o _outlier_, se houver, está em somente uma das extremidades da distribuição).

Abaixo criamos algumas funções que nos auxiliam nos cálculos e na exibição dos resultados:

In [ ]:
def grubb_test(g, n, alpha=0.05, tailed='two-tailed'):
  if tailed == 'two-tailed':
    critical = ((n - 1)/sqrt(n)) * sqrt(sct.t.isf(alpha/(2*n), n-2)**2/(n - 2 + sct.t.isf(alpha/(2*n), n-2)**2))
    
    return (g, critical, g > critical)
  elif tailed == 'one-tailed':
    critical = ((n - 1)/sqrt(n)) * sqrt(sct.t.isf(alpha/(n), n-2)**2/(n - 2 + sct.t.isf(alpha/(n), n-2)**2))
    
    return (g, critical, g > critical)
  else:
    raise ValueError(f"Invalid tailed argument")

In [ ]:
def grubb_summary(result, decimals=10):
  return (
    f"Null hypothesis: there is no outliers in the data set\n"
    f"Test statistic: {np.round(result[0], decimals)}, "
    f"Grubb's critical value: {np.round(result[1], decimals)}, "
    f"Reject: {result[2]}"
  )

In [ ]:
def next_outlier_candidate(data):
  sample_distances = (data - data.mean()).abs()
  candidate_idx = sample_distances.idxmax()
  candidate_value = data[candidate_idx]
  candidate_statistic = sample_distances.max()/data.std()
  
  return (candidate_idx, candidate_value, candidate_statistic, len(data))

Ao executarmos o teste de Grubb no nosso conjunto de alturas, encontramos alguns valores onde a hipótese nula é rejeitada, ou seja, há evidência de que o valor extremo é um _outlier_.

In [ ]:
height_outlier_grubb = height_outlier.copy()
outliers_grubb = pd.Series()
has_outlier = True

while has_outlier:
  outlier_candidate = next_outlier_candidate(height_outlier_grubb)

  print(f"Index: {outlier_candidate[0]}, "
        f"Value: {np.round(outlier_candidate[1], 3)}, "
        f"Test statistic: {np.round(outlier_candidate[2], 3)}, "
        f"Sample size: {outlier_candidate[3]}\n")

  result = grubb_test(outlier_candidate[2], outlier_candidate[3])

  print(grubb_summary(result, 3))

  has_outlier = result[2]

  if has_outlier:
    height_outlier_grubb = height_outlier_grubb.drop(index=outlier_candidate[0])
    outliers_grubb.at[outlier_candidate[0]] = outlier_candidate[1]
    
  print(f"\n")

In [ ]:
outliers_grubb

Abaixo comparamos os _outliers_ encontrados por cada um dos três métodos:

In [ ]:
outliers = pd.Series({"IQR": outliers_iqr.index.values,
                      "Normal": outliers_dist.index.values,
                      "Grubb": outliers_grubb.index.values})

outliers.apply(np.sort)

## _Features_ de texto

Dados textuais são muito ricos e muito fáceis de serem encontrados. Diversos _data sets_ são compostos por documentos textuais e ainda um simples _scrapper_ pode coletar dezenas de milhares de documentos da Internet. Coleções de documentos são frequentemente chamadas de _corpus_ (plural, _corpora_).

Nosso objetivo aqui é somente mostrar como preprocessar de forma simples _features_ textuais. Para isso, utilizaremos o _data set_ 20 newsgroups, que contém milhares de documentos categorizados em 20 grupos (desde astronomia até carros).

Abaixo escolhemos somente três grupos para restringir nosso escopo:

In [ ]:
categories = ["sci.crypt", "sci.med", "sci.space"]

newsgroups = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

Temos agora um _corpus_ com 1782 documentos:

In [ ]:
len(newsgroups.data)

Um exemplo de documento desse _corpus_ é mostrado abaixo:

In [ ]:
document_idx = 4
documents_total = len(newsgroups.data)

print(f"> Document {document_idx} of {documents_total}:\n\n{newsgroups.data[document_idx]}")
print(f"> Category: {newsgroups.target_names[newsgroups.target[document_idx]]}")

Quando trabalhando com dados textuais, uma representação simples é ter:

* Cada documento em uma linha.
* Cada palavra (ou termo) em uma coluna.

Por exemplo, se nosso vocábulário (conjunto de todas palavras ou termos do _corpus_) tiver tamanho 10000 e tivermos 100 documentos, então nosso _data set_ será composto de 100 linhas e 10000 colunas.

O valor de cada célula, $x_{i, j}$, (interseção da linha $i$ com a coluna $j$) do _data set_ depende da tranformação que aplicarmos.

A transformação mais simples é a contagem de palavras no documento, ou seja, $x_{i, j}$ indica o número de ocorrências da palavra $j$ no documento $i$.

Isso pode ser obtido no sklearn pelo `CountVectorizer`:

In [ ]:
count_vectorizer = CountVectorizer()
newsgroups_counts = count_vectorizer.fit_transform(newsgroups.data)

In [ ]:
type(newsgroups_counts)

Abaixo escolhemos dez palavras contidas no _corpus_ para exemplificar:

In [ ]:
words_idx = sorted([count_vectorizer.vocabulary_.get(f"{word.lower()}") for word in
                    [u"clipper", u"Kapor",
                     u"monitor", u"gibberish",
                     u"Banks", u"private",
                     u"study", u"group",
                     u"Colorado", u"Business"]])

pd.DataFrame(newsgroups_counts[:5, words_idx].toarray(), columns=np.array(count_vectorizer.get_feature_names())[words_idx])

Por exemplo, o valor 2 na interseção do documento 0 com a coluna `clipper` indica que a palavra _clipper_ aparece duas vezes no documento 0. Obviamente é possível que uma mesma palavra apareça em múltiplos documentos e mais óbvio ainda que um documento contenha múltiplas palavras.

O problema com essa abordagem é que não temos como medir relevância dos termos. E se o  termo é super comum e aparece em quase todos documentos? E se o termo aparece muitas vezes no mesmo documento, mas poucas vezes nos outros?

Essas perguntas não podem ser respondidas simplesmente com a contagem de termos acima. Para isso, precisamos do tf-idf.

O tf-idf é uma estatística baseada no _corpus_ composta de outras duas estatísticas:

* $\text{tf}(t, d)$, ou _term frequency_, é uma medida de quantas vezes o termo $t$ aparece no documento $d$. Algumas opções estão disponíveis, mas a mais simples é a contagem do número de ocorrências do termo no documento, $f_{t, d}$, exatamente o que computamos acima. Essa é a forma como sklearn define $tf$:

$$\text{tf}(t, d) = f_{t, d}$$

* $\text{idf}(t)$, ou _inverse document frequency_, é uma medida de relevância do termo em todos documentos do _corpus_. O sklearn a computa, seguindo valores _default_, da seguinte forma:

$$\text{idf}(t) = \log{\frac{1+n}{1 + d_{t}}} + 1$$

onde $n$ é o número de documentos no _corpus_ e $d_{t}$ é o número de documentos no _corpus_ que contêm o termo $t$ ($0 < d_{t} \leq n$).

O tf-idf é calculado multiplicando esses dois valores:

$$\text{tf-idf}(t, d) = \text{tf}(t, d) \times \text{idf}(t) = f_{t, d} \times \log{\frac{1+n}{1 + d_{t}}} + 1$$

O sklearn também normaliza todos documentos resultantes, ou seja todas linhas da matriz, para terem norma unitária. Em outras palavras, os elementos do vetor de tf-idf do documento $i$ são dados por:

$$\text{tf-idf}(i, j)_{\text{normalizado}} = \frac{\text{tf-idf}(i, j)}{\sqrt{\text{tf-idf}(i, 1)^{2} + \text{tf-idf}(i, 2)^{2} + \cdots + \text{tf-idf}(i, T)^{2}}}$$

onde $T$ é o número de termos do _corpus_, ou seja, o tamanho do vocabulário.

O tf-idf é sempre um valor não negativo e quanto mais alto, maior a relevância do termo.

Note como o tf aumenta de acordo com o número de ocorrências do termo no documento: quanto mais frequente o termo, mas relevante ele parece ser.

O idf é uma medida de "raridade" do termo através de todo _corpus_: quanto mais alto, menos o termo aparece no _corpus_ e consequentemente mais informação ele traz.

Multiplicando os dois, temos uma medida do quão relevante aquele termo é para aquele documento no _corpus_.

O sklearn provê um transformador, `TfidfTransformer`, que transforma de uma matriz de frequências, como a retornada pelo `CountVectorizer`, e retorna uma matriz de tf-idf:

In [ ]:
tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(newsgroups_counts)

newsgroups_tfidf = tfidf_transformer.transform(newsgroups_counts)

In [ ]:
pd.DataFrame(newsgroups_tfidf[:5, words_idx].toarray(), columns=np.array(count_vectorizer.get_feature_names())[words_idx])

Também podemos obter a matriz de tf-idf diretamente do _corpus_ sem ter que passar pela matriz de frequência com o transformador `TfidfVectorizer`:

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(newsgroups.data)

newsgroups_tfidf_vectorized = tfidf_vectorizer.transform(newsgroups.data)

In [ ]:
pd.DataFrame(newsgroups_tfidf_vectorized[:5, words_idx].toarray(), columns=np.array(count_vectorizer.get_feature_names())[words_idx])

Note como a matriz acima é exatamente igual a retornada pelo `TfidfTransformer`.

O resultado (igual da matriz de frequência) é um _data set_ com 1782 documentos e 33796 termos:

In [ ]:
newsgroups_tfidf_vectorized.shape

## Referências

* [Feature engineering](https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html)

* [Feature Scaling with scikit-learn](http://benalexkeen.com/feature-scaling-with-scikit-learn/)

* [Anthony Goldbloom gives you the secret to winning Kaggle competitions](https://www.import.io/post/how-to-win-a-kaggle-competition/)

* [What are some best practices in Feature Engineering?](https://www.quora.com/What-are-some-best-practices-in-Feature-Engineering)

* [Discover Feature Engineering, How to Engineer Features and How to Get Good at It](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/)

* [Fundamental Techniques of Feature Engineering for Machine Learning](https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114)

* [Feature Engineering Cookbook for Machine Learning](https://medium.com/@michaelabehsera/feature-engineering-cookbook-for-machine-learning-7bf21f0bcbae)

* [A Simple Guide to Scikit-learn Pipelines](https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf)

* [Outlier detection with Scikit Learn](https://www.mikulskibartosz.name/outlier-detection-with-scikit-learn/)

* [Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

* [WTF is TF-IDF?](https://www.kdnuggets.com/2018/08/wtf-tf-idf.html)
